# Process the Customers vs Orders Synapse Link Data

In [40]:
# Load the SynapseLink Customers and Orders data into a Dataframes.
# Then, filter the Orders for just the 'order' doctype.

df_customers = spark.read\
    .format("cosmos.olap")\
    .option("spark.synapse.linkedService", "demoCosmosDB")\
    .option("spark.cosmos.container", "customers")\
    .load()

df_orders = spark.read\
    .format("cosmos.olap")\
    .option("spark.synapse.linkedService", "demoCosmosDB")\
    .option("spark.cosmos.container", "orders")\
    .load()

df_order_docs = df_orders.filter(df_orders["doctype"].isin(["order"]))

print('df_customers, shape: {} x {}'.format(
        df_customers.count(), len(df_customers.columns)))
df_customers.printSchema()

print('df_orders, shape: {} x {}'.format(
        df_orders.count(), len(df_orders.columns)))
df_orders.printSchema()

print('df_order_docs, shape: {} x {}'.format(
        df_order_docs.count(), len(df_order_docs.columns)))



StatementMeta(poolspark3s, 43, 39, Finished, Available)

df_customers, shape: 100000 x 13
root
 |-- _rid: string (nullable = true)
 |-- _ts: long (nullable = true)
 |-- pk: string (nullable = true)
 |-- doctype: string (nullable = true)
 |-- customerId: string (nullable = true)
 |-- name: string (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- address: struct (nullable = true)
 |    |-- street: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- zip: string (nullable = true)
 |-- doc_epoch: long (nullable = true)
 |-- doc_time: string (nullable = true)
 |-- id: string (nullable = true)
 |-- _etag: string (nullable = true)

df_orders, shape: 1049182 x 21
root
 |-- _rid: string (nullable = true)
 |-- _ts: long (nullable = true)
 |-- pk: long (nullable = true)
 |-- doctype: string (nullable = true)
 |-- orderId: long (nullable = true)
 |-- lineNumber: long (nullable = true)
 |-- customerId: string (nullable = true)
 |-- sku: long (nu

In [41]:
# Display the first few rows of the df_customers Dataframe

display(df_customers.limit(3))


StatementMeta(poolspark3s, 43, 40, Finished, Available)

SynapseWidget(Synapse.DataFrame, 804cd0ef-ecb9-49e5-aff8-c1c2a0a6aedf)

In [42]:
# Display the first few rows of the df_order_docs Dataframe

display(df_order_docs.limit(3))

StatementMeta(poolspark3s, 43, 41, Finished, Available)

SynapseWidget(Synapse.DataFrame, 41480927-68ca-4fa5-8ac6-eeb088c51382)

In [43]:
# Create Narrower/Minimal Dataframes for the Join operation 

from pyspark.sql.functions import col

df_customers_minimal = df_customers.select(
    col('customerId'),
    col('name'))

print('df_customers_minimal, shape: {} x {}'.format(
        df_customers_minimal.count(), len(df_customers_minimal.columns)))
df_customers_minimal.printSchema()

df_orders_minimal = df_order_docs.select(
    col('orderId'),
    col('customerId'),
    col('item_count'),
    col('order_total'))

print('df_orders_minimal, shape: {} x {}'.format(
        df_orders_minimal.count(), len(df_orders_minimal.columns)))
df_orders_minimal.printSchema()

StatementMeta(poolspark3s, 43, 42, Finished, Available)

df_customers_minimal, shape: 100000 x 2
root
 |-- customerId: string (nullable = true)
 |-- name: string (nullable = true)

df_orders_minimal, shape: 300000 x 4
root
 |-- orderId: long (nullable = true)
 |-- customerId: string (nullable = true)
 |-- item_count: long (nullable = true)
 |-- order_total: double (nullable = true)

In [51]:
# Join the (narrow) Customers to their (narrow) Order documents

df_joined = df_orders_minimal.join(df_customers_minimal, ['customerId']) \
    .sort("customerId", ascending=False)


print('df_joined, shape: {} x {}'.format(
        df_joined.count(), len(df_joined.columns)))
df_joined.printSchema()


StatementMeta(poolspark3s, 43, 50, Finished, Available)

df_joined, shape: 300000 x 5
root
 |-- customerId: string (nullable = true)
 |-- orderId: long (nullable = true)
 |-- item_count: long (nullable = true)
 |-- order_total: double (nullable = true)
 |-- name: string (nullable = true)

In [52]:
# Display the first few rows of the df_joined Dataframe

display(df_joined.limit(20))

StatementMeta(poolspark3s, 43, 51, Finished, Available)

SynapseWidget(Synapse.DataFrame, 57bbb019-736a-4afd-bc37-85719f4ee721)

In [53]:
# Group the df_joined Dataframe by customerId, sum on order total and total_orders

df_grouped = df_joined.groupby("customerId") \
    .sum("order_total").alias('total_orders') \
    .sort("customerId", ascending=False)

display(df_grouped.printSchema())
print((df_grouped.count(), len(df_grouped.columns)))



StatementMeta(poolspark3s, 43, 52, Finished, Available)

root
 |-- customerId: string (nullable = true)
 |-- sum(order_total): double (nullable = true)

(95119, 2)

In [54]:
import pyspark.sql.functions as F 

df_agg = df_joined.groupBy("customerId") \
    .agg(
        F.count("customerId").alias('order_count'), \
        F.sum("order_total").alias("total_dollar_amount"), \
        F.sum("item_count").alias("total_item_count")) \
        .sort("customerId", ascending=False)

display(df_agg.printSchema())
print((df_agg.count(), len(df_agg.columns)))

StatementMeta(poolspark3s, 43, 53, Finished, Available)

root
 |-- customerId: string (nullable = true)
 |-- order_count: long (nullable = false)
 |-- total_dollar_amount: double (nullable = true)
 |-- total_item_count: long (nullable = true)

(95119, 4)

In [55]:
display(df_agg.limit(30))

StatementMeta(poolspark3s, 43, 54, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2b8e2eae-7172-4558-ae6d-4dd7b2e29654)